In [1]:
import pandas as pd

In [2]:
customers = pd.read_csv('../../processed_data/customers.csv')
types = pd.read_csv('../../processed_data/lda_distns.csv')
transaction = pd.read_csv('../../processed_data/database.csv')

In [3]:
names = [str(i) for i in range(25)]
types.columns = ['account_id']+names
types.head()

,account_id,0,1,2,3,4,5,6,7,8,...,15,16,17,18,19,20,21,22,23,24
0,576,0.118368,0.000212,0.007477,0.085284,0.005008,0.000212,0.012573,0.000212,0.168622,...,0.020308,0.042442,0.000212,0.081874,0.003604,0.116314,0.085687,0.181844,0.000212,0.003120
1,704,0.000859,0.027608,0.000860,0.000859,0.000859,0.000859,0.019092,0.000859,0.000860,...,0.000859,0.000859,0.000859,0.218398,0.000859,0.000859,0.188473,0.000859,0.000859,0.000860
2,192,0.000543,0.000543,0.392860,0.000543,0.000543,0.000543,0.000543,0.007720,0.000543,...,0.000543,0.000543,0.000543,0.249266,0.000543,0.000543,0.204132,0.000543,0.000543,0.014677
3,10364,0.001191,0.001191,0.459827,0.001191,0.001191,0.001191,0.001191,0.001191,0.001191,...,0.001191,0.001191,0.001191,0.220206,0.001191,0.001191,0.001191,0.001191,0.001191,0.001191
4,497,0.000412,0.019238,0.032279,0.105024,0.390893,0.000411,0.000412,0.015450,0.000412,...,0.000412,0.000412,0.000412,0.210518,0.000412,0.000412,0.000412,0.000412,0.000412,0.000411


In [4]:
customers.head()

,Unnamed: 0,account_id,district_id,frequency,date,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,0,576,55,0,1993-01-01,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
1,1,704,55,0,1993-01-01,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
2,2,192,55,0,1993-01-08,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
3,3,10364,55,0,1993-01-17,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894
4,4,497,55,0,1993-04-15,Brno - venkov,south Moravia,157042,49,70,18,0,9,33.9,8743,1.88,2.43,111,3659,3894


In [19]:
data = pd.merge(customers, types, on='account_id')[['account_id', 'A4', 'A11', 'A13', 'A16']+names]

In [20]:
data.shape

(4499, 30)

In [21]:
transaction.head()

,Unnamed: 0,account_id,trans_id,category,payee,k-symbol,date,amount,mean_income,Initial Balance
0,0,1,5,0,1288,-1,1995-03-24,122.633333,1358.788889,33.333333
1,1,1,199,29,1072,-1,1995-04-13,420.000000,1358.788889,33.333333
2,2,1,3530438,40,911,2,1995-04-23,0.640000,1358.788889,33.333333
3,3,1,6,0,381,-1,1995-04-30,122.633333,1358.788889,33.333333
4,4,1,200,46,180,-1,1995-05-13,70.000000,1358.788889,33.333333


In [22]:
# Need only mean income from the transactions table
calc_income = transaction[['account_id', 'mean_income']]
prevWarn    = pd.options.mode.chained_assignment
pd.options.mode.chained_assignment = None  # default='warn' --> ignore in place warning
calc_income.drop_duplicates(inplace=True)
pd.options.mode.chained_assignment = prevWarn
print(calc_income.shape)
calc_income.head()

(4500, 2)


,account_id,mean_income
0,1,1358.788889
238,2,7191.931004
715,3,2902.271667
831,4,2084.311702
1016,5,2062.000000


In [23]:
data = pd.merge(data, calc_income, on='account_id', how='left')[['account_id', 'mean_income','A4', 'A11', 'A13', 'A16']+names]

In [24]:
data.shape

(4499, 31)

In [25]:
data['A16'] = data['A16']/data['A4']

In [30]:
# Not sure what this is for...
if False:
    data = data[data['amount']>0]

In [129]:
x = data[['mean_income','A4','A11','A13','A16']].as_matrix()
y = data[names].as_matrix()

In [130]:
print(x.shape, y.shape)

(4499, 5) (4499, 25)


In [131]:
data[['mean_income','A4','A11','A13','A16']].head(10)
#data[['A4','A11','A13','A16']].drop_duplicates()  # >> only 77 unique combinations of these 4 variables across 4500.

,mean_income,A4,A11,A13,A16
0,3072.195270,157042,8743,2.43,0.024796
1,9367.563265,157042,8743,2.43,0.024796
2,2066.260870,157042,8743,2.43,0.024796
3,7579.751176,157042,8743,2.43,0.024796
4,8185.426733,157042,8743,2.43,0.024796
5,14655.634101,157042,8743,2.43,0.024796
6,8256.462312,157042,8743,2.43,0.024796
7,3472.171176,157042,8743,2.43,0.024796
8,2735.714286,157042,8743,2.43,0.024796
9,4889.432510,157042,8743,2.43,0.024796


In [132]:
data[names].head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,0.118368,0.000212,0.007477,0.085284,0.005008,0.000212,0.012573,0.000212,0.168622,0.000212,...,0.020308,0.042442,0.000212,0.081874,0.003604,0.116314,0.085687,0.181844,0.000212,0.003120
1,0.000859,0.027608,0.000860,0.000859,0.000859,0.000859,0.019092,0.000859,0.000860,0.000859,...,0.000859,0.000859,0.000859,0.218398,0.000859,0.000859,0.188473,0.000859,0.000859,0.000860
2,0.000543,0.000543,0.392860,0.000543,0.000543,0.000543,0.000543,0.007720,0.000543,0.000543,...,0.000543,0.000543,0.000543,0.249266,0.000543,0.000543,0.204132,0.000543,0.000543,0.014677
3,0.001191,0.001191,0.459827,0.001191,0.001191,0.001191,0.001191,0.001191,0.001191,0.001191,...,0.001191,0.001191,0.001191,0.220206,0.001191,0.001191,0.001191,0.001191,0.001191,0.001191
4,0.000412,0.019238,0.032279,0.105024,0.390893,0.000411,0.000412,0.015450,0.000412,0.000412,...,0.000412,0.000412,0.000412,0.210518,0.000412,0.000412,0.000412,0.000412,0.000412,0.000411


In [133]:
np.sum(y,0)/4499


array([ 0.02274455,  0.00967676,  0.06122619,  0.02933991,  0.02409954,
        0.09842417,  0.0293924 ,  0.01161045,  0.02527735,  0.01901109,
        0.02367406,  0.01231438,  0.015658  ,  0.06343827,  0.07337881,
        0.03240909,  0.02277001,  0.04177651,  0.10678996,  0.01366191,
        0.04463884,  0.11720082,  0.08587615,  0.00692295,  0.00868782])

In [134]:
y2 = np.copy(y)
for ii in range(y.shape[0]):
    y[ii,] = np.random.multinomial(1, y2[ii,])

In [137]:
y[1:10,]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
      

In [144]:
# Now we build MLP:
# =======================

import numpy, sys, json, hashlib, time

import keras
import keras.models, keras.layers.recurrent
from keras.layers.core import Dense
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.normalization import BatchNormalization
from six.moves import range
from six import print_
from sklearn.preprocessing import StandardScaler

input_dim       = 5
nSamples        = x.shape[0]
output_dim      = y.shape[1]
mid_level_width = 100

# Dummy inputs for testing...
#x=numpy.random.uniform(size=(nSamples,input_dim))
#y=numpy.zeros((nSamples,output_dim))

nn              =keras.models.Sequential()
nn.add(Dense(mid_level_width,input_shape=(input_dim,),activation="relu"))
# nn.add(Dense(mid_level_width,activation="relu"))
nn.add(Dense(output_dim,activation="re"))
#op              = keras.optimizers.rmsprop(lr=1)
op              = keras.optimizers.Adam(lr=0.0008, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

nn.compile(loss='mean_squared_error', optimizer=op, metrics = ["accuracy"])
nn.summary()

scaler = StandardScaler()
scaled_x = scaler.fit_transform(x)

nn.fit(scaled_x,y2,nb_epoch=500,verbose=1,shuffle=0,batch_size=134)
print ("exiting at "+ time.strftime("%Y-%m-%d %H:%M:%S"))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_117 (Dense)            (None, 100)               600       
_________________________________________________________________
dense_118 (Dense)            (None, 25)                2525      
Total params: 3,125.0
Trainable params: 3,125
Non-trainable params: 0.0
_________________________________________________________________


/Users/alexbird/anaconda3/lib/python3.5/site-packages/keras/models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/500
4499/4499 [==============================] - 0s - loss: 0.0073 - acc: 0.1218     
Epoch 2/500
4499/4499 [==============================] - 0s - loss: 0.0068 - acc: 0.1374     
Epoch 3/500
4499/4499 [==============================] - 0s - loss: 0.0066 - acc: 0.1165     
Epoch 4/500
4499/4499 [==============================] - 0s - loss: 0.0065 - acc: 0.1111     
Epoch 5/500
4499/4499 [==============================] - 0s - loss: 0.0065 - acc: 0.1131     
Epoch 6/500
4499/4499 [==============================] - 0s - loss: 0.0065 - acc: 0.1218     
Epoch 7/500
4499/4499 [==============================] - 0s - loss: 0.0065 - acc: 0.1218     
Epoch 8/500
4499/4499 [==============================] - 0s - loss: 0.0064 - acc: 0.1269     
Epoch 9/500
4499/4499 [==============================] - 0s - loss: 0.0064 - acc: 0.1263     
Epoch 10/500
4499/4499 [==============================] - 0s - loss: 0.0064 - acc: 0.1320     
Epoch 11/500
4499/4499 [==============================] - 0

In [ ]:
nn.predict(x)

In [120]:
from sklearn.ensemble import ExtraTreesClassifier
rforest=ExtraTreesClassifier(n_estimators=10,max_depth=6)
category_y2 = np.argmax(y2,1)
print(category_y2[:13])
rforest.fit(x,np.argmax(y2,1))
sum(rforest.predict(x)==category_y2)


(4499,)
[22 10  2  2  4 18 22 18 20 23 13  4 18]


955

In [145]:
pred = nn.predict(x)
print(pred)

[[ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]


In [146]:
import numpy as np 
pred     = np.asarray(pred)
uq_array = set([tuple(row) for row in pred])
uq_array

{(0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (0.0,
  2.2767542e-36,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (0.0,
  1.1039425e-31,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (0.0,
  2.0310674e-29,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (0.0,
  7.0912386e-27,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0),
 (0.0,
  1.4985097e-25,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,


list